# Volume Generation
This notebook explains a method to create volume mesh for both bones and cartilages

# imports and user-defined properties

In [1]:
import numpy as np
import meshplot as mp
from pathlib import Path
import os
import igl
import matplotlib.pyplot as plt
import wildmeshing as wm
import sys

sys.path.append(str(Path.home()/'Documents'/'Github'/'libhip'))
sys.path.append('../')

import src

INPUT SUBJECT INFORMATION:
- subject_id = choose between m1 -> m11 to select one of the detaset in the repository.
- user-defined names: these names belong to input bone models. 'l' and 'r' refer to the 'left' and the 'right' side of the body.
- i_dim, o_dim: the input and output dimension ( 'mm' = millimeters, 'm' = meters )
- o_format: the format you want the files to be save at ( '.obj' , '.stl' )
- eps: the envelope of size epsilon. Using smaller envelope preserves features better but also takes longer time. 
  The default value of epsilon is b/1000, where b is the length of the diagonal of the bounding box. 
  Please refer to https://github.com/wildmeshing/fTetWild for more information.
- l_* : the ideal edge length. Using smaller ideal edge length gives a denser mesh but also takes longer time. 
  The default ideal edge length is b/20.
  Please refer to https://github.com/wildmeshing/fTetWild for more information.

In [2]:
# do you want coarse or fine mesh?
coarse = True

if coarse:
    density = '_coarse'
else:
    density = '_fine'

# subject id
subject_id = 'm1'  
folder_tag = subject_id + density

# Are you cartilage geometries full or implicit models?
full_model = True

# dimensions 
i_dim  = 'mm'     
o_dim  = 'mm'

# file format and suffix
i_format = '.obj'
o_format = '.msh'

DIRECTORIES and PATHS:
- main_dir: locate this directory to where the libhip repository is cloned to.
- ftetwild_dir: locate this path to the 'build' folder of ftetwild.
- i_dir:  locate this directory to the input bone surface mesh files.
- o_dir:  locate this directory to where you. want the cleaned and remeshed bone models to be saved at.

In [3]:
ftetwild_dir = Path.home()/ 'Documents'/ 'Github'/ 'fTetWild'/ 'build'
main_dir     = Path('..')

i_dir        = main_dir/ 'model_generation'/ 'cargen_output'/ subject_id
json_o_dir   = main_dir/ 'model_generation'/ 'json_output'  / folder_tag
ftet_o_dir   = main_dir/ 'model_generation'/ 'ftet_output'  / folder_tag
o_dir        = main_dir/ 'model_generation'/ 'volgen_output'/ folder_tag
   
config_name  = subject_id + '_config'
config_path  = str((main_dir/ 'config'/ config_name).with_suffix(".yml"))
config       = src.Config (config_path)

# Remove all files inside output directory if it exists, otherwise create it
if json_o_dir.is_dir():
    for file in json_o_dir.iterdir():
        if file.is_file():
            file.unlink()
else:
    json_o_dir.mkdir(exist_ok=False)
    
if ftet_o_dir.is_dir():
    for file in ftet_o_dir.iterdir():
        if file.is_file():
            file.unlink()
else:
    ftet_o_dir.mkdir(exist_ok=False)
    
if o_dir.is_dir():
    for file in o_dir.iterdir():
        if file.is_file():
            file.unlink()
else:
    o_dir.mkdir(exist_ok=False)

In [4]:
# input paths - bones
input_bone_tag = subject_id + '_cg_bone_'

input_sacrum  = input_bone_tag + 'sacrum'
input_lhip    = input_bone_tag + 'lhip'  
input_rhip    = input_bone_tag + 'rhip'
input_lfemur  = input_bone_tag + 'lfemur'
input_rfemur  = input_bone_tag + 'rfemur'

input_sacrum_path = str((i_dir/ input_sacrum).with_suffix(i_format).resolve())
input_lhip_path   = str((i_dir/ input_lhip).with_suffix(i_format).resolve())
input_rhip_path   = str((i_dir/ input_rhip).with_suffix(i_format).resolve())
input_lfemur_path = str((i_dir/ input_lfemur).with_suffix(i_format).resolve())
input_rfemur_path = str((i_dir/ input_rfemur).with_suffix(i_format).resolve())

In [5]:
# input paths - joints
if full_model:
    input_joint_tag = subject_id + '_cg_jnt_'
else:
    input_joint_tag = subject_id + '_cg_jnt_implicit_'
    
# sacroiliac joint
lsj = input_joint_tag + 'lsi'
rsj = input_joint_tag + 'rsi'

# pubic joint
pj  = input_joint_tag + 'pj'

# sacroiliac and pubic joint paths
input_lsj_path = str((i_dir/ lsj).with_suffix(i_format).resolve())
input_rsj_path = str((i_dir/ rsj).with_suffix(i_format).resolve())
input_pj_path  = str((i_dir/ pj) .with_suffix(i_format).resolve())

# hip joint with gap
lhj_ac_wg = input_joint_tag + 'lh_ac_w_gap' 
lhj_fc_wg = input_joint_tag + 'lh_fc_w_gap' 
rhj_ac_wg = input_joint_tag + 'rh_ac_w_gap' 
rhj_fc_wg = input_joint_tag + 'rh_fc_w_gap'  

# hip joint without gap
lhj_ac_wog = input_joint_tag + 'lh_ac_wo_gap' 
lhj_fc_wog = input_joint_tag + 'lh_fc_wo_gap'
rhj_ac_wog = input_joint_tag + 'rh_ac_wo_gap'
rhj_fc_wog = input_joint_tag + 'rh_fc_wo_gap'

# hip joint w/wo gap paths
input_lhj_ac_wg_path = str((i_dir/ lhj_ac_wg).with_suffix(i_format).resolve())
input_rhj_ac_wg_path = str((i_dir/ rhj_ac_wg).with_suffix(i_format).resolve())
input_lhj_fc_wg_path = str((i_dir/ lhj_fc_wg).with_suffix(i_format).resolve())
input_rhj_fc_wg_path = str((i_dir/ rhj_fc_wg).with_suffix(i_format).resolve())

input_lhj_ac_wog_path = str((i_dir/ lhj_ac_wog).with_suffix(i_format).resolve())
input_rhj_ac_wog_path = str((i_dir/ rhj_ac_wog).with_suffix(i_format).resolve())
input_lhj_fc_wog_path = str((i_dir/ lhj_fc_wog).with_suffix(i_format).resolve())
input_rhj_fc_wog_path = str((i_dir/ rhj_fc_wog).with_suffix(i_format).resolve())

In [6]:
# ftetwild output directories - with gap
ftet_output_tag = subject_id + '_csg_'

ftet_output_pg_wg   = ftet_output_tag + '_girdle_w_gap'
ftet_output_legL_wg = ftet_output_tag + '_left_leg_w_gap' 
ftet_output_legR_wg = ftet_output_tag + '_right_leg_w_gap'

ftet_output_pg_wg_dir   = str((ftet_o_dir/ ftet_output_pg_wg).resolve())
ftet_output_legL_wg_dir = str((ftet_o_dir/ ftet_output_legL_wg).resolve())
ftet_output_legR_wg_dir = str((ftet_o_dir/ ftet_output_legR_wg).resolve())

# ftetwild output directories - without gap
ftet_output_pg_wog   = ftet_output_tag + '_girdle_wo_gap'
ftet_output_legL_wog = ftet_output_tag + '_left_leg_wo_gap' 
ftet_output_legR_wog = ftet_output_tag + '_right_leg_wo_gap'

ftet_output_pg_wog_dir   = str((ftet_o_dir/ ftet_output_pg_wog).resolve())
ftet_output_legL_wog_dir = str((ftet_o_dir/ ftet_output_legL_wog).resolve())
ftet_output_legR_wog_dir = str((ftet_o_dir/ ftet_output_legR_wog).resolve())

In [7]:
# volgen output paths - with gap
output_pg_wg   = subject_id + '_girdle_w_gap'   + density
output_legL_wg = subject_id + '_left_leg_w_gap' + density
output_legR_wg = subject_id + '_right_leg_w_gap'+ density

output_pg_wg_dir   = str(o_dir/ output_pg_wg)
output_legL_wg_dir = str(o_dir/ output_legL_wg)
output_legR_wg_dir = str(o_dir/ output_legR_wg)

# volgen output paths - without gap
output_pg_wog   = subject_id + '_girdle_wo_gap'   + density
output_legL_wog = subject_id + '_left_leg_wo_gap' + density
output_legR_wog = subject_id + '_right_leg_wo_gap'+ density

output_pg_wog_dir   = str(o_dir/ output_pg_wog)
output_legL_wog_dir = str(o_dir/ output_legL_wog)
output_legR_wog_dir = str(o_dir/ output_legR_wog)

# implementation

In [8]:
# epsilon and edge length for the femur (l_leg) and the sacrum and the hip (l_girdle)
config_vol = config.vol_var

if coarse:
    eps = str(config_vol.epsilon_coarse)
    l_leg = str(config_vol.edge_length_leg_coarse)
    l_girdle = str(config_vol.edge_length_girdle_coarse)
else:
    eps = str(config_vol.epsilon_fine)
    l_leg = str(config_vol.edge_length_leg_fine)
    l_girdle = str(config_vol.edge_length_girdle_fine)

In [9]:
s_vertices, s_faces   = src.read (input_sacrum_path, i_dim) 
lh_vertices, lh_faces = src.read (input_lhip_path, i_dim)
rh_vertices, rh_faces = src.read (input_rhip_path, i_dim) 
lf_vertices, lf_faces = src.read (input_lfemur_path, i_dim)
rf_vertices, rf_faces = src.read (input_rfemur_path, i_dim) 

lsj_vertices, lsj_faces = src.read (input_lsj_path, i_dim)
rsj_vertices, rsj_faces = src.read (input_rsj_path, i_dim)
pj_vertices,  pj_faces  = src.read (input_pj_path,  i_dim)

# with gap
lhj_ac_wg_vertices, lhj_ac_wg_faces = src.read (input_lhj_ac_wg_path, i_dim )
rhj_ac_wg_vertices, rhj_ac_wg_faces = src.read (input_rhj_ac_wg_path, i_dim )

lhj_fc_wg_vertices, lhj_fc_wg_faces = src.read (input_lhj_fc_wg_path, i_dim )
rhj_fc_wg_vertices, rhj_fc_wg_faces = src.read (input_rhj_fc_wg_path,i_dim )

# without gap
lhj_ac_wog_vertices, lhj_ac_wog_faces = src.read (input_lhj_ac_wog_path, i_dim )
rhj_ac_wog_vertices, rhj_ac_wog_faces = src.read (input_rhj_ac_wog_path, i_dim )

lhj_fc_wog_vertices, lhj_fc_wog_faces = src.read (input_lhj_fc_wog_path, i_dim )
rhj_fc_wog_vertices, rhj_fc_wog_faces = src.read (input_rhj_fc_wog_path,i_dim )

number of faces after reading 46456
number of faces after reading 35726
number of faces after reading 35528
number of faces after reading 24908
number of faces after reading 25108
number of faces after reading 4088
number of faces after reading 3230
number of faces after reading 616
number of faces after reading 3576
number of faces after reading 2600
number of faces after reading 8208
number of faces after reading 8672
number of faces after reading 3576
number of faces after reading 2600
number of faces after reading 8208
number of faces after reading 8672


In [10]:
frame = mp.plot( s_vertices, s_faces, c = src.bone, shading = src.sh_false )
frame.add_mesh ( lh_vertices, lh_faces, c = src.bone, shading = src.sh_false )
frame.add_mesh ( rh_vertices, rh_faces, c = src.bone, shading = src.sh_false )
frame.add_mesh ( lf_vertices, lf_faces, c = src.bone, shading = src.sh_false )
frame.add_mesh ( rf_vertices, rf_faces, c = src.bone, shading = src.sh_false )
frame.add_mesh ( lhj_ac_wg_vertices, lhj_ac_wg_faces, c = src.pastel_light_blue, shading = src.sh_false )
frame.add_mesh ( lhj_fc_wg_vertices, lhj_fc_wg_faces, c = src.pastel_light_blue, shading = src.sh_false )
frame.add_mesh ( rhj_ac_wg_vertices, rhj_ac_wg_faces, c = src.pastel_light_blue, shading = src.sh_false )
frame.add_mesh ( rhj_fc_wg_vertices, rhj_fc_wg_faces, c = src.pastel_light_blue, shading = src.sh_false )
frame.add_mesh ( lsj_vertices, lsj_faces, c = src.pastel_light_blue, shading = src.sh_false )
frame.add_mesh ( rsj_vertices, rsj_faces, c = src.pastel_light_blue, shading = src.sh_false )
frame.add_mesh ( pj_vertices, pj_faces, c = src.pastel_light_blue, shading = src.sh_false)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-4.006748…

11

## the pelvic girdle

### with gap

In [11]:
# csg operation
op_0      = src.mk_union (input_lhip_path, input_lhj_ac_wg_path)
op_1      = src.mk_union (input_rhip_path, input_rhj_ac_wg_path)
op_2      = src.mk_union (op_0, input_lsj_path)
op_3      = src.mk_union (op_1, input_rsj_path)
op_4      = src.mk_union (op_2, op_3)
op_5      = src.mk_union (op_4, input_sacrum_path)
op_girdle = src.mk_union (op_5, input_pj_path)

# make the json file
json_output_girdle_wg_path = src.mk_json(ftet_output_pg_wg, op_girdle, json_o_dir)

# create the volume mesh
src.run_boolean(ftetwild_dir, json_output_girdle_wg_path, ftet_output_pg_wg_dir, eps, l_girdle)

In [12]:
src.girdle_filter (ftet_output_pg_wg_dir, output_pg_wg_dir,
               lsj_vertices, lsj_faces, rsj_vertices, rsj_faces, 
               lhj_ac_wg_vertices, lhj_ac_wg_faces, rhj_ac_wg_vertices, rhj_ac_wg_faces,
               pj_vertices, pj_faces, s_vertices, s_faces,
               lh_vertices, lh_faces, rh_vertices, rh_faces,
               i_dim, o_dim)

[1 2 3 4 5 6 7 8]
length of elements one 2286
length of elements two 1759
length of elements three 2247
length of elements four 1477
length of elements five 624
length of elements six 48294
length of elements seven 29811
length of elements eight 29017
length of labels 115515


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-2.399223…

### without gap

In [13]:
# csg operation
op_0      = src.mk_union (input_lhip_path, input_lhj_ac_wog_path)
op_1      = src.mk_union (input_rhip_path, input_rhj_ac_wog_path)
op_2      = src.mk_union (op_0, input_lsj_path)
op_3      = src.mk_union (op_1, input_rsj_path)
op_4      = src.mk_union (op_2, op_3)
op_5      = src.mk_union (op_4, input_sacrum_path)
op_girdle = src.mk_union (op_5, input_pj_path)

# make the json file
json_output_girdle_wog_path = src.mk_json(ftet_output_pg_wog, op_girdle, json_o_dir)

# create the volume mesh
src.run_boolean(ftetwild_dir, json_output_girdle_wog_path, ftet_output_pg_wog_dir, eps, l_girdle)

In [14]:
src.girdle_filter (ftet_output_pg_wog_dir, output_pg_wog_dir,
               lsj_vertices, lsj_faces, rsj_vertices, rsj_faces, 
               lhj_ac_wog_vertices, lhj_ac_wog_faces, rhj_ac_wog_vertices, rhj_ac_wog_faces,
               pj_vertices, pj_faces, s_vertices, s_faces,
               lh_vertices, lh_faces, rh_vertices, rh_faces,
               i_dim, o_dim)

[1 2 3 4 5 6 7 8]
length of elements one 2081
length of elements two 1727
length of elements three 2113
length of elements four 1474
length of elements five 650
length of elements six 48490
length of elements seven 28755
length of elements eight 28923
length of labels 114213


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-2.382347…

## left femur & left femoral cartilage

### with gap

In [15]:
# csg operation 
op_legL = src.mk_union (input_lfemur_path, input_lhj_fc_wg_path)

# make the json file
json_output_legL_wg_path = src.mk_json (ftet_output_legL_wg, op_legL, json_o_dir)

# create the volume mesh
src.run_boolean (ftetwild_dir, json_output_legL_wg_path, ftet_output_legL_wg_dir, eps, l_leg)

In [16]:
src.leg_filter (ftet_output_legL_wg_dir, output_legL_wg_dir,
                lhj_fc_wg_vertices, lhj_fc_wg_faces,
                lf_vertices, lf_faces,
                i_dim, o_dim)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(103.40033…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(103.41011…

number of physical elements [1 2]
length of elements 1 6383
length of elements 2 34862
length of labels 41245


### without gap

In [17]:
# csg operation 
op_legL = src.mk_union (input_lfemur_path, input_lhj_fc_wog_path)

# make the json file
json_output_legL_wog_path = src.mk_json(ftet_output_legL_wog, op_legL, json_o_dir)

# create the volume mesh
src.run_boolean (ftetwild_dir, json_output_legL_wog_path, ftet_output_legL_wog_dir, eps, l_leg)

In [18]:
src.leg_filter (ftet_output_legL_wog_dir, output_legL_wog_dir,
                lhj_fc_wog_vertices, lhj_fc_wog_faces,
                lf_vertices, lf_faces,
                i_dim, o_dim)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(103.34125…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(103.34288…

number of physical elements [1 2]
length of elements 1 6298
length of elements 2 35564
length of labels 41862


## right femur & right femoral cartilage

### with gap

In [19]:
# csg operation 
op_legR = src.mk_union(input_rfemur_path, input_rhj_fc_wg_path)

# make the json file
json_output_legR_wg_path = src.mk_json(ftet_output_legR_wg, op_legR, json_o_dir)

# create the volume mesh
src.run_boolean (ftetwild_dir, json_output_legR_wg_path, ftet_output_legR_wg_dir, eps, l_leg)

In [20]:
src.leg_filter (ftet_output_legR_wg_dir, output_legR_wg_dir,
                rhj_fc_wg_vertices, rhj_fc_wg_faces,
                rf_vertices, rf_faces,
                i_dim, o_dim)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-102.1604…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-102.1265…

number of physical elements [1 2]
length of elements 1 6668
length of elements 2 35480
length of labels 42148


### without gap

In [21]:
# csg operation 
op_legR = src.mk_union(input_rfemur_path, input_rhj_fc_wog_path)

# make the json file
json_output_legR_wog_path = src.mk_json(ftet_output_legR_wog, op_legR, json_o_dir)

# create the volume mesh
src.run_boolean (ftetwild_dir, json_output_legR_wog_path, ftet_output_legR_wog_dir, eps, l_leg)

In [22]:
src.leg_filter (ftet_output_legR_wog_dir, output_legR_wog_dir,
                rhj_fc_wog_vertices, rhj_fc_wog_faces,
                rf_vertices, rf_faces,
                i_dim, o_dim )

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-101.9957…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-101.9937…

number of physical elements [1 2]
length of elements 1 6532
length of elements 2 36295
length of labels 42827


## make sure of the gaps

In [23]:
# # with gap
# lfc_w_gap_verices, lfc_w_gap_faces = src.read(vg_output_left_w_gap_path+ '_cart.obj', i_dim)
# lpc_w_gap_verices, lpc_w_gap_faces = src.read(vg_output_nord_w_gap_path+ '_lpc.obj', i_dim)
# rfc_w_gap_verices, rfc_w_gap_faces = src.read(vg_output_right_w_gap_path+ '_cart.obj', i_dim)
# rpc_w_gap_verices, rpc_w_gap_faces = src.read(vg_output_nord_w_gap_path+ '_rpc.obj', i_dim)

# lsd_w_gap, _, _ = igl.signed_distance(lfc_w_gap_verices,lpc_w_gap_verices, lpc_w_gap_faces, return_normals=False)
# rsd_w_gap, _, _ = igl.signed_distance(rfc_w_gap_verices,rpc_w_gap_verices, rpc_w_gap_faces, return_normals=False)

# print('minimum distance in the left hip joint with gap',np.round(np.min(lsd_w_gap),2))
# print('minimum distance in the right hip joint with gap',np.round(np.min(rsd_w_gap),2))

# # without gap
# lfc_wo_gap_verices, lfc_wo_gap_faces = src.read(vg_output_left_wo_gap_path+ '_cart.obj', i_dim)
# lpc_wo_gap_verices, lpc_wo_gap_faces = src.read(vg_output_nord_wo_gap_path+ '_lpc.obj', i_dim)
# rfc_wo_gap_verices, rfc_wo_gap_faces = src.read(vg_output_right_wo_gap_path+ '_cart.obj', i_dim)
# rpc_wo_gap_verices, rpc_wo_gap_faces = src.read(vg_output_nord_wo_gap_path+ '_rpc.obj', i_dim)

# lsd_wo_gap, _, _ = igl.signed_distance(lfc_wo_gap_verices,lpc_wo_gap_verices, lpc_wo_gap_faces, return_normals=False)
# rsd_wo_gap, _, _ = igl.signed_distance(rfc_wo_gap_verices,rpc_wo_gap_verices, rpc_wo_gap_faces, return_normals=False)

# print('minimum distance in the left hip joint without gap',np.round(np.min(lsd_wo_gap),2))
# print('minimum distance in the right hip joint without gap',np.round(np.min(rsd_wo_gap),2))